# Pipeline de Súmulas do STJ

Notebook para extrair súmulas do site do STJ, pré-processar os verbetes, gerar embeddings com SentenceTransformers e indexá-los com FAISS para busca semântica.

Este notebook contém passos para coleta (web scraping), preparação de dados, geração de embeddings, construção do índice e exemplos de consulta.

## Coleta (web scraping)

Este bloco utiliza Selenium para navegar no site do STJ e coletar número e verbete de cada súmula. Os resultados são salvos em `sumulas_metadata.json`.

In [24]:
import json
import re
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [25]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
service = Service("/usr/local/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

In [26]:
driver.get("https://scon.stj.jus.br/SCON/sumstj/toc.jsp?documentosSelecionadosParaPDF=&numDocsPagina=100&tipo_visualizacao=&data=&p=true&b=SUMU&thesaurus=JURIDICO&i=1&l=100&ordenacao=-%40NUM&operador=e")

sumulas = []
while True:
    for grid in driver.find_elements(By.CLASS_NAME, "gridSumula"):
        try:
            num = grid.find_element(By.CLASS_NAME, "numeroSumula").text
            verbete = grid.find_element(By.CLASS_NAME, "blocoVerbete").text
            sumulas.append({"numero": num, "verbete": verbete})
        except Exception:
            continue
    try:
        botao_proxima = driver.find_element(By.CLASS_NAME, "iconeProximaPagina")
        botao_proxima.click()
        WebDriverWait(driver, 10).until(EC.staleness_of(grid))
    except Exception:
        break

with open("sumulas_metadata.json", "w", encoding="utf-8") as f:
    json.dump(sumulas, f, ensure_ascii=False, indent=2)


## Geração de embeddings e indexação

Aqui carregamos os verbetes extraídos, geramos embeddings usando `sentence-transformers` e criamos um índice FAISS (IndexFlatIP). Os vetores são normalizados com L2 antes de gravar o índice.

In [ ]:
with open("sumulas_metadata.json", "r", encoding="utf-8") as f:
    sumulas = json.load(f)

verbetes = [s["verbete"] for s in sumulas]
embeddings = model.encode(verbetes, convert_to_numpy=True)
embeddings = np.asarray(embeddings, dtype=np.float32)
faiss.normalize_L2(embeddings)

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

faiss.write_index(index, "sumulas_index.faiss")